# [Issue# 1897 Additional Visuals for PUC Analysis](https://github.com/cal-itp/data-analyses/issues/1897)

Received list of transit operators cohorts that may be exempt from efficiency reporting, per PUC 99314.11, .6 and .7. 
- create visuals based on grouping set by list
- recreate visuals based on previous notebook work

## [99314.6](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?sectionNum=99314.6.&lawCode=PUC)
>`funds shall be allocated for operating or capital purpose` pursuant to Sections 99313 and 99314 to an operator `if the operator meets either of the following efficiency standards`:
>- (A) `The operator shall receive its entire allocation`, and any or all of this allocation may be used for operating purposes, if the operator’s `total operating cost per revenue vehicle hour` in the latest year for which audited data are available `does not exceed the sum of the preceding year’s total operating cost per revenue vehicle hour and an amount equal to the product of the percentage change in the Consumer Price Index for the same period multiplied by the preceding year’s total operating cost per revenue vehicle hour.`
>- (B) The operator shall receive its entire allocation, and any or all of this allocation may be used for operating purposes, `if the operator’s average total operating cost per revenue vehicle hour` in the latest three years for which audited data are available `does not exceed the sum of the average of the total operating cost per revenue vehicle hour in the three years preceding the latest year for which audited data are available and an amount equal to the product of the average percentage change in the Consumer Price Index for the same period multiplied by the average total operating cost per revenue vehicle hour in the same three years`.
## [99314.7 (mainly MTC specific)](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?lawCode=PUC&sectionNum=99314.7.)
>the `Metropolitan Transportation Commission` shall apply the following eligibility standards to the operators within the region subject to its jurisdiction:

# [99314.11](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?sectionNum=99314.11.&nodeTreePath=17.11.2.8&lawCode=PUC)
>`Sections 99314.6 and 99314.7 do not apply to an operator for a fiscal year in which the operator expended from local funding an amount for transit operations not less than the amount the operator expended from local funding for transit operations during the 2018–19 fiscal year.` As used in this subdivision, “local funding” means any nonstate grant funds or other revenues generated by, earned by, or distributed to, an operator.

Meaning, if a transit operator spent local funds >= the local funds spent during FY 2018-2019, they are exempt from meeting efficiency standards(?)

In [9]:
import pandas as pd
import altair as alt
from functools import cache
from calitp_data_analysis.gcs_pandas import GCSPandas
from calitp_data_analysis.sql import get_engine, to_snakecase, query_sql

@cache
def gcs_pandas():
    return GCSPandas()

# Read in cohort list data

In [2]:
cohort_data = gcs_pandas().read_csv("gs://calitp-analytics-data/data-analyses/ntd/fbr_local_funding_by_cohorts_2019-2024_compiled.csv")

In [34]:
cohort_data.columns = cohort_data.columns.str.lower()
cohort_data["ntd_id"] = cohort_data["ntd_id"].astype("str")

In [35]:
display(
    cohort_data.info(),
    cohort_data.head()
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1797 entries, 0 to 1796
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ntd_id       1797 non-null   object
 1   urban_rural  1797 non-null   object
 2   cohort       1797 non-null   object
 3   metric       1797 non-null   object
 4   year         1797 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 70.3+ KB


None

,ntd_id,urban_rural,cohort,metric,year
0,91018,Rural,Group A,Farebox Recovery Ratio,2019
1,91062,Rural,Group A,Farebox Recovery Ratio,2019
2,91088,Rural,Group A,Farebox Recovery Ratio,2019
3,91043,Rural,Group A,Farebox Recovery Ratio,2019
4,91036,Rural,Group A,Farebox Recovery Ratio,2019


In [5]:
cohort_data.value_counts(
    subset=["urban_rural","metric","cohort","year"]
)

urban_rural  metric                          cohort   year
Urban        Farebox Recovery Ratio          Group B  2023    52
                                                      2024    52
                                                      2019    51
                                                      2020    51
             Local Funding % Change vs 2019  Group B  2024    50
                                                              ..
Rural        Farebox Recovery Ratio          Group A  2023    12
                                                      2021    12
                                             Group C  2024    12
                                             Group A  2024    11
                                                      2022    11
Name: count, Length: 72, dtype: int64

# Read in analysis data from prev notebook

In [6]:
gcs_path = "gs://calitp-analytics-data/data-analyses/ntd/"
ntd_name = "puc_analysis_data.parquet"
ntd_analysis_data = gcs_pandas().read_parquet(f"{gcs_path}{ntd_name}")

In [7]:
ntd_analysis_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4152 entries, 0 to 4151
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   ntd_id             4002 non-null   object  
 1   source_agency      4152 non-null   object  
 2   agency_status      4152 non-null   object  
 3   primary_uza_name   4152 non-null   object  
 4   uza_population     4152 non-null   int64   
 5   uza_area_sq_miles  4152 non-null   float64 
 6   year               4152 non-null   int64   
 7   mode               4152 non-null   object  
 8   service            4152 non-null   object  
 9   reporter_type      4152 non-null   object  
 10  total_vrh          2623 non-null   float64 
 11  total_upt          2623 non-null   float64 
 12  total_pmt          2623 non-null   float64 
 13  ntd_id_2022        3798 non-null   object  
 14  county             3798 non-null   object  
 15  caltrans_district  3798 non-null   object  
 16  _merge     

In [8]:
ntd_analysis_data["year"].unique()

array([2018, 2019, 2020, 2021, 2022, 2023])

# May need to requery this data to include 2024
is 2024 NTD data in the warehouse now? copy pasted from initial puc analysis notebook.

In [10]:
metric_list = [
    "pmt",
    "upt",
    "vrh",
    # "opexp_total" # not needed for this project
]

# empty list for appending DFs
df_list = []

# loop to query pmt, upt and vrh from 2018 to 2024
for metric in metric_list:
        query = f"""
        SELECT
          ntd_id,
          source_agency,
          agency_status,
          primary_uza_name,
          uza_population,
          uza_area_sq_miles,
          year,
          mode,
          type_of_service,
          reporter_type,
          SUM({metric}) AS total_{metric},
        FROM
          `cal-itp-data-infra.mart_ntd_funding_and_expenses.fct_service_data_and_operating_expenses_time_series_by_mode_{metric}`
        WHERE
          source_state = "CA"
          AND year BETWEEN 2018 AND 2024
        GROUP BY
          ntd_id,
          source_agency,
          agency_status,
          primary_uza_name,
          uza_population,
          uza_area_sq_miles,
          year,
          mode,
          type_of_service,
          reporter_type
        """
        # create df
        metric = query_sql(query, as_df=True)

        # append df to list
        df_list.append(metric)

# unpack list into separate DFs
ntd_pmt, ntd_upt, ntd_vrh = df_list

In [19]:
# ensure 2024 data is in 
ntd_pmt["year"].value_counts()

year
2019    557
2021    557
2020    557
2018    557
2024    557
2022    557
2023    557
Name: count, dtype: int64

In [21]:
display( 
    ntd_upt.head(3)
)

,ntd_id,source_agency,agency_status,primary_uza_name,uza_population,uza_area_sq_miles,year,mode,type_of_service,reporter_type,total_upt
0,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.8,2019,MG,PT,Full Reporter,886515.0
1,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.8,2018,MB,PT,Full Reporter,NaN
2,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.8,2020,MB,PT,Full Reporter,NaN


## merge all the metrics together

In [22]:
merge_on_col = [
    "ntd_id",
    "year",
    "source_agency",
    "agency_status",
    "primary_uza_name",
    "uza_population",
    "uza_area_sq_miles",
    "mode",
    "type_of_service",
    "reporter_type",
]

merge_1 = ntd_vrh.merge(ntd_upt, on=merge_on_col, how="inner")
# merge_2 = merge_1.merge(ntd_vrh, on=merge_on_col, how = "inner")

ntd_metrics_merge = merge_1.merge(ntd_pmt, on=merge_on_col, how="inner")

In [23]:
ntd_metrics_merge.head(3)

,ntd_id,source_agency,agency_status,primary_uza_name,uza_population,uza_area_sq_miles,year,mode,type_of_service,reporter_type,total_vrh,total_upt,total_pmt
0,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.8,2019,MG,PT,Full Reporter,19815.0,886515.0,2819118.0
1,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.8,2019,MB,PT,Full Reporter,NaN,NaN,NaN
2,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.8,2021,MG,PT,Full Reporter,17819.0,112981.0,359280.0


## get districts for ntd ID
- Do i still need district data for this specific analysis?

In [24]:
for metric in metric_list:
        query = f"""
        SELECT
          `mart_transit_database.dim_organizations`.`key` AS `key`,
          `mart_transit_database.dim_organizations`.`source_record_id` AS `source_record_id`,
          `mart_transit_database.dim_organizations`.`name` AS `name`,
          `mart_transit_database.dim_organizations`.`ntd_id_2022` AS `ntd_id_2022`,
          `Bridge_Organizations_X_Headquarters_County_Geography___Key`.`county_geography_name` AS `county`,
          `Dim_County_Geography___County_Geography_Key`.`caltrans_district` AS `caltrans_district`
        FROM
          `mart_transit_database.dim_organizations`

        LEFT JOIN `mart_transit_database.bridge_organizations_x_headquarters_county_geography` AS `Bridge_Organizations_X_Headquarters_County_Geography___Key` ON `mart_transit_database.dim_organizations`.`key` = `Bridge_Organizations_X_Headquarters_County_Geography___Key`.`organization_key`
          LEFT JOIN `mart_transit_database.dim_county_geography` AS `Dim_County_Geography___County_Geography_Key` ON `Bridge_Organizations_X_Headquarters_County_Geography___Key`.`county_geography_key` = `Dim_County_Geography___County_Geography_Key`.`key`
        WHERE
          (
            `mart_transit_database.dim_organizations`.`_is_current` = TRUE
          )

           AND (
            `mart_transit_database.dim_organizations`.`ntd_id_2022` IS NOT NULL
          )
          AND (
            (
              `mart_transit_database.dim_organizations`.`ntd_id_2022` <> ''
            )

            OR (
              `mart_transit_database.dim_organizations`.`ntd_id_2022` IS NULL
            )
          )
          AND (
            `Bridge_Organizations_X_Headquarters_County_Geography___Key`.`_is_current` = TRUE
          )
          AND (
            `Dim_County_Geography___County_Geography_Key`.`_is_current` = TRUE
          )
        """
        # create df
        ntd_id_x_district = query_sql(query, as_df=True)
        
ntd_id_x_district["caltrans_district"] = ntd_id_x_district["caltrans_district"].astype("str")

In [25]:
ntd_id_x_district.head()

,key,source_record_id,name,ntd_id_2022,county,caltrans_district
0,d84a961daa618c733f9d9c3bd49c322f,recJtH0Ae8YNo01aj,Access Services,90157,Los Angeles,7
1,9b5971d16d58e4fcafa694ee7fa33b12,rec79AM4tMwdokWhE,Alpine County,91116,Alpine,10
2,e5de5083d68e8c2463a784ceb13e91f2,recUTSH4TT1wB3RSC,Attentive Transportation LLC,90314,Sacramento,3
3,957618c89db2f5e992caa5ca2e6086ab,rec7F6JKLMVrRhQJU,Bishop Paiute Tribe,99268,Inyo,9
4,a024fabd0002f9c9bd636042de30715d,recE6qJFuoREa9EHg,Calaveras County,91063,Calaveras,10


## merge the ntd metrics with Caltrans Districts

In [26]:
ntd_metrics_merge = ntd_metrics_merge.merge(
    ntd_id_x_district[["ntd_id_2022","county","caltrans_district"]],
    left_on = "ntd_id",
    right_on = "ntd_id_2022",
    how="inner",
    indicator=True
)

In [28]:
ntd_metrics_merge.head()

,ntd_id,source_agency,agency_status,primary_uza_name,uza_population,uza_area_sq_miles,year,mode,type_of_service,reporter_type,total_vrh,total_upt,total_pmt,ntd_id_2022,county,caltrans_district,_merge
0,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.8,2019,MG,PT,Full Reporter,19815.0,886515.0,2819118.0,90003,San Francisco,4,both
1,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.8,2019,MB,PT,Full Reporter,NaN,NaN,NaN,90003,San Francisco,4,both
2,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.8,2021,MG,PT,Full Reporter,17819.0,112981.0,359280.0,90003,San Francisco,4,both
3,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.8,2021,DR,PT,Full Reporter,NaN,NaN,NaN,90003,San Francisco,4,both
4,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.8,2019,YR,DO,Full Reporter,41924.0,2225056.0,15283299.0,90003,San Francisco,4,both


# merge ntd metrics with exempty data
- merge on ntd_id
- are there any unmerged rows?

In [33]:
cohort_data.dtypes

ntd_id          int64
urban_rural    object
cohort         object
metric         object
year            int64
dtype: object

In [32]:
ntd_metrics_merge.dtypes

ntd_id                 object
source_agency          object
agency_status          object
primary_uza_name       object
uza_population          int64
uza_area_sq_miles     float64
year                    int64
mode                   object
type_of_service        object
reporter_type          object
total_vrh             float64
total_upt             float64
total_pmt             float64
ntd_id_2022            object
county                 object
caltrans_district      object
_merge               category
dtype: object

In [49]:
ntd_cohort_merge = ntd_metrics_merge.drop(columns="_merge").merge(
    cohort_data,
    left_on = ["ntd_id","year"],
    right_on = ["ntd_id","year"],
    indicator= True,
)

In [50]:
ntd_cohort_merge.head()

,ntd_id,source_agency,agency_status,primary_uza_name,uza_population,uza_area_sq_miles,year,mode,type_of_service,reporter_type,total_vrh,total_upt,total_pmt,ntd_id_2022,county,caltrans_district,urban_rural,cohort,metric,_merge
0,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.8,2019,MG,PT,Full Reporter,19815.0,886515.0,2819118.0,90003,San Francisco,4,Urban,Group A,Farebox Recovery Ratio,both
1,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.8,2019,MG,PT,Full Reporter,19815.0,886515.0,2819118.0,90003,San Francisco,4,Urban,Group B,Local Funding % Change vs 2019,both
2,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.8,2019,MB,PT,Full Reporter,NaN,NaN,NaN,90003,San Francisco,4,Urban,Group A,Farebox Recovery Ratio,both
3,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.8,2019,MB,PT,Full Reporter,NaN,NaN,NaN,90003,San Francisco,4,Urban,Group B,Local Funding % Change vs 2019,both
4,90003,San Francisco Bay Area Rapid Transit District ...,Active,"San Francisco--Oakland, CA",3515933,513.8,2021,MG,PT,Full Reporter,17819.0,112981.0,359280.0,90003,San Francisco,4,Urban,Group B,Farebox Recovery Ratio,both


In [71]:
ntd_metrics_merge.shape

(3801, 17)

In [51]:
# any unmerged rows? NONE
ntd_cohort_merge["_merge"].value_counts()

_merge
both          4965
left_only        0
right_only       0
Name: count, dtype: int64

In [74]:
ntd_cohort_merge["metric"].unique()

array(['Farebox Recovery Ratio', 'Local Funding % Change vs 2019'],
      dtype=object)

In [ ]:
# Sanity check
# pick up a couple of NTD ID, see if the merge data tracks with the cohort data
sample_ids = ntd_cohort_merge["ntd_id"].sample(3).to_list()
keep_cols=[
    "ntd_id",
    "source_agency",
    "mode",
    "type_of_service",
    "total_vrh",
    "total_pmt",
    "total_upt",
    "urban_rural",
    "cohort",
    "metric",
    "year"
]

for sample_id in sample_ids:
    display(
        f"Sameple NTD ID: {sample_id}",
        "cohort data",
        cohort_data[
            (cohort_data["ntd_id"]== sample_id)
            & (cohort_data["year"].isin([2023,2024]))
            ].sort_values(by=["urban_rural","cohort","metric","year"]).head(5),
        "merge table",
        ntd_cohort_merge[
            (ntd_cohort_merge["ntd_id"]== sample_id)
            & (ntd_cohort_merge["year"].isin([2023,2024]))
            ][keep_cols].sort_values(by=["urban_rural","cohort","metric","year"]),
        
    )

# cohort data matches, 
# looks a little weird since the ntd metrics is per mode and TOS. the cohort data becomes categorical. GTG

In [91]:
# need to separate list by both metrics (farebox and funding change)
cohort_merge_farebox = ntd_cohort_merge[ntd_cohort_merge["metric"]=="Farebox Recovery Ratio"]
cohort_merge_funding = ntd_cohort_merge[ntd_cohort_merge["metric"]=="Local Funding % Change vs 2019"]

In [94]:
display(
    cohort_merge_farebox.shape,
    cohort_merge_funding.shape,
    cohort_merge_farebox["metric"].unique(),
    cohort_merge_funding["metric"].unique(),
)

(2460, 20)

(2505, 20)

array(['Farebox Recovery Ratio'], dtype=object)

array(['Local Funding % Change vs 2019'], dtype=object)